In [1]:
from itertools import islice
from datetime import datetime
import json
import pandas as pd
import numpy as np
from pprint import pprint

np.set_printoptions(precision=3, suppress=True)

In [2]:
files = "./ml-20m/"
movies = pd.read_csv(files + "movies.csv", index_col="movieId")
ratings = pd.read_csv(files + "ratings.csv", index_col=["movieId", "userId"])

In [3]:
movies.genres = movies.genres.str.split("|")

In [4]:
R = ratings.rating
movies["rating"] = R.mean(level="movieId")

In [5]:
ratings.timestamp = ratings.timestamp.apply(datetime.fromtimestamp)

In [43]:
m1, m2 = movies.ix[318], movies.ix[858]
r1 = R.ix[m1.name] - m1.rating
r2 = R.ix[m2.name] - m2.rating
similarity = (r1 * r2).sum(skipna=True) / np.sqrt((r1 * r1).sum() * (r2 * r2).sum())
similarity

0.099149146678717062

In [8]:
"""
m1 = movies.iloc[318]
m2 = movies.iloc[858] 
users = np.intersect1d(m1.userId, m2.userId, assume_unique=True)
rows = ["rating"] * len(users)
r1 = ratings_index.lookup([(m1.name, u) for u in users], rows) - m1.rating
r2 = ratings_index.lookup([(m2.name, u) for u in users], rows) - m2.rating
(r1 * r2).sum() / np.sqrt((r1 * r1).sum() * (r2 * r2).sum())
"""

'\nm1 = movies.iloc[318]\nm2 = movies.iloc[858] \nusers = np.intersect1d(m1.userId, m2.userId, assume_unique=True)\nrows = ["rating"] * len(users)\nr1 = ratings_index.lookup([(m1.name, u) for u in users], rows) - m1.rating\nr2 = ratings_index.lookup([(m2.name, u) for u in users], rows) - m2.rating\n(r1 * r2).sum() / np.sqrt((r1 * r1).sum() * (r2 * r2).sum())\n'

In [9]:
R = np.array(["rating" for _ in range(100000)])

def sim_pearson(m1, m2, m_index=lambda x: x.name):
    users = np.intersect1d(m1.userId, m2.userId, assume_unique=True)
    if users.size == 0:
        return 0.0
    r1 = ratings_index.lookup([(m1.Index, u) for u in users], R[:len(users)]) - m1.rating
    r2 = ratings_index.lookup([(m_index(m2), u) for u in users], R[:len(users)]) - m2.rating
    count = (r1 * r2).sum()
    if count == 0:
        return 0.0
    return count / np.sqrt((r1 * r1).sum() * (r2 * r2).sum())

R

array(['rating', 'rating', 'rating', ..., 'rating', 'rating', 'rating'], 
      dtype='<U6')

In [10]:
def similarities(movie, measure=sim_pearson, m_index=lambda x: x.Index):
    return np.array([
        measure(m, movie, m_index) for m in movies[:m_index(movie)].itertuples()
    ])

In [14]:
m = movies.iloc[318]
tuple(m)

('Strawberry and Chocolate (Fresa y chocolate) (1993)',
 ['Drama'],
 3.761930585683297,
 array([   112,    310,    330,    569,    903,   1105,   1198,   1313,
          1610,   1931,   1983,   2127,   2354,   2932,   3013,   3109,
          3136,   3184,   3249,   3344,   3352,   3515,   3594,   3628,
          3797,   3890,   3915,   4024,   4110,   4256,   4450,   4509,
          4825,   4976,   5110,   5248,   5561,   5698,   5743,   5784,
          5967,   6082,   6133,   6166,   6168,   6356,   6406,   6442,
          6465,   6764,   6892,   6979,   7266,   7292,   7472,   7499,
          7555,   7563,   7640,   7738,   8003,   8119,   8249,   8344,
          8398,   8400,   8405,   8634,   8647,   8963,   9291,   9394,
          9511,   9574,   9768,  10027,  10365,  10412,  10581,  10624,
         10651,  10659,  10951,  11231,  11253,  11292,  11325,  11593,
         11641,  11761,  11891,  11929,  12053,  12126,  12629,  12901,
         13026,  13170,  13181,  13222,  13255, 

In [17]:
def similarities2(movie, measure=sim_pearson, m_index=lambda x: x.Index):
    return np.array([
        measure(m, movie, m_index) for m in movies.itertuples()
    ])

SyntaxError: invalid syntax (<ipython-input-17-e8047613ce91>, line 6)

In [18]:
%timeit similarities2(m, m_index=lambda x: x.name)

1 loop, best of 3: 5.79 s per loop


In [ ]:
#similarities(list(islice(movies.itertuples(), 1))[0])
sims = sc.parallelize(islice(movies.itertuples(), 2000), 4).map(similarities).collect()

In [32]:
max_len = sims[-1].size
result = np.zeros((max_len, max_len))
for i, s in enumerate(sims):
    result[i, :s.size] = s
result + result.T - np.identity(max_len)

array([[ 1.        ,  0.31501184,  0.17379116,  0.16582331,  0.21226605,
         0.11603759,  0.19424208,  0.19642781,  0.08660063,  0.22551062,
         0.20889393, -0.00774983,  0.23755015,  0.1103709 ,  0.14167647,
         0.08702847,  0.18640497,  0.00199259,  0.14468261,  0.08724871],
       [ 0.31501184,  1.        ,  0.35424204,  0.30434227,  0.42471435,
         0.11666153,  0.34585337,  0.40833376,  0.38390436,  0.29971012,
         0.28986993,  0.29343352,  0.36508444,  0.12953916,  0.44822586,
         0.13384564,  0.17119933,  0.1845611 ,  0.3075455 ,  0.40804445],
       [ 0.17379116,  0.35424204,  1.        ,  0.27102355,  0.47317664,
         0.15431559,  0.30378027,  0.47393198,  0.30958607,  0.27860627,
         0.2925893 ,  0.30834273,  0.36490975,  0.08996912,  0.3289103 ,
         0.14294103,  0.09119123,  0.18655605,  0.29257432,  0.39560433],
       [ 0.16582331,  0.30434227,  0.27102355,  1.        ,  0.34759068,
         0.1616584 ,  0.29106905,  0.49372458,  

In [12]:
#sims = rdd.collect() # aggregate("", lambda acc, x: "{}\n{}".format(acc, json.dumps(x)), lambda x, y: "{}\iin{}".format(x, y))